In [2]:
# inference with NPE
from evo_models import WF
from inference_utils import get_prior
from sbi.inference import NPE, NLE, NRE, NPSE
from sbi.utils.user_input_checks import (
    check_sbi_inputs,
    process_prior,
    process_simulator,
)

import torch
import pickle

# Define the prior
prior = get_prior()

# Define the simulator
def simulator(theta):
    s = 10**theta[0].item()
    mu = 10**theta[1].item()
    N = int(1e8)
    G = 200
    res = WF(s, mu, N, G)
    return torch.tensor(res)

# inference
# Check prior, return PyTorch prior.
prior, num_parameters, prior_returns_numpy = process_prior(prior)

# Check simulator, returns PyTorch simulator able to simulate batches.
simulator = process_simulator(simulator, prior, prior_returns_numpy)

# Consistency check after making ready for sbi.
check_sbi_inputs(simulator, prior)

num_simulations = 100_000
theta = prior.sample((num_simulations,))
x = simulator(theta)


KeyboardInterrupt



In [9]:
torch.save(theta, "test_sims/theta.pt")
torch.save(x, "test_sims/x.pt")

In [1]:
import numpy as np
import pandas as pd
import torch
from sbi.utils import BoxUniform
from inference_utils import get_prior, get_dist
from evo_models import WF, WF_bottleneck, WF_DFE

# Define the prior and constants
prior = get_prior()
N = int(1e8)
G = 200

# Define the simulator
def simulator(model, theta):
    s = 10**theta[0].item()
    mu = 10**theta[1].item()
    if model == 'WF':
        res = WF(s, mu, N, G)
    elif model == 'WF_bottleneck':
        bottlenecks = [10*(i+1) for i in range(G//10)]
        N_bottleneck = N//1000
        bot_dict = {b: N_bottleneck for b in bottlenecks}
        res = WF_bottleneck(s, mu, N, G, bot_dict, seed=None) # Run the bottleneck model
    elif model == 'WF_DFE':
        dist = get_dist(s)
        res = WF_DFE(mu, N, G, dist)
    return torch.tensor(res)

def export_sims(model, num_simulations):
    x_export = torch.empty(num_simulations, G//10)
    thetas = prior.sample((num_simulations,))
    for i in range(len(thetas)):
        theta = thetas[i]
        x_export[i,:] = simulator(model, theta)
    torch.save(x_export,f'test_sims/test_x_{model}.pt')
    torch.save(thetas,f'test_sims/test_theta_{model}.pt')

if __name__ == '__main__':
    export_sims('WF', 1000)
    export_sims('WF_bottleneck', 1000)
    export_sims('WF_DFE', 1000)